# Try to read and convert the networks

For the Sioux network

In [6]:
import csv
import pandas as pd
import numpy as np

# reading a TNTP text file
def read_file_net(network_name, suffix="net"):
    assert suffix in ["net","node","trips", "flow"]
    with open("data/" + network_name + "/" + network_name + "_" + suffix + ".tntp") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        index_to_legend = {}
        data = {}
        metadata = {}
        line_count = 0
        for row in csv_reader:
            if len(row)==0:
                # The row is empty 
                continue 
            if "<" in row[0]:
                metadata_aux = row[0].split('<')[1].split('> ')
                if metadata_aux[0]=="END OF METADATA>":
                    continue
                metadata[metadata_aux[0]] = metadata_aux[1]
            else:
                if (suffix=="net" or suffix=="node" or suffix=="flow") and row[-1] == ";":
                    if line_count == 0:
                        # the first line is the legend
                        i = 0
                        for legend in row[:-1]:
                            # print(legend)
                            index_to_legend[i] = legend
                            data[legend] = []
                            i += 1
                    else:
                        i = 0
                        for point in row[:-1]:
                            data[index_to_legend[i]].append(point)
                            i += 1
                        # print(row)
                    line_count += 1
                elif(suffix=="trips"):
                    if row[0] == 'Origin ':
                        origin = int(row[1])
                        data[origin] = {}
                    else:
                        # print(row)
                        dest_array = row[0].split(";")
                        # print(dest_array)
                        for dest in dest_array[:-1]:
                            dest_tmp = dest.split(":")
                            data[origin][int(dest_tmp[0])] = float(dest_tmp[1])
                else:
                    print("This setence should never appear. Error")
    d = pd.DataFrame(data)
    return d, metadata

In [21]:
# net and node are ok
network_name = "Braess"

d, metadata = read_file_net(network_name, 'net')
d_net = d.drop(['~ ',  'Length ', 'Speed limit ', 'Toll ', 'Type'], axis=1)

#d_node, metadata_aux = read_file_net(network_name, 'node')
#metadata.update(metadata_aux)

d_od, metadata_aux = read_file_net(network_name, 'trips')
# d_od is a matrix
metadata.update(metadata_aux)

#d_sol, metadata_aux = read_file_net(network_name, 'flow')
#metadata.update(metadata_aux)

This setence should never appear. Error
This setence should never appear. Error


In [22]:
print(d_net)
# print(d_sol.head())

  Init node  Term node  Capacity  Free Flow Time            B Power
0          1          3         1      0.00000001  1000000000     1
1          1          4         1              50        0.02     1
2          3          2         1              50        0.02     1
3          3          4         1              10         0.1     1
4          4          2         1      0.00000001  1000000000     1


In [23]:
result = pd.concat([d_net, d_sol], axis=1)
result['Init node '] = pd.to_numeric(result['Init node '])
result['From '] = pd.to_numeric(result['From '])
result['Term node '] = pd.to_numeric(result['Term node '])
result['To '] = pd.to_numeric(result['To '])
result['Cost '] = pd.to_numeric(result['Cost '])
result['Free Flow Time '] = pd.to_numeric(result['Free Flow Time '])
result['Volume Capacity '] = pd.to_numeric(result['Volume Capacity '])
result['Capacity '] = pd.to_numeric(result['Capacity '])
result['B'] = pd.to_numeric(result['B'])
result['Power'] = pd.to_numeric(result['Power'])

# print(list(result.columns.values))
if np.sum((result['Init node '] == result['From ']).values) == result.shape[0]:
    result = result.drop('From ', axis=1)
if np.sum((result['Term node '] == result['To ']).values) == result.shape[0]:
    result = result.drop('To ', axis=1)

# result['cost test'] = result['Free Flow Time '].values * (1 + result['B'].values * ((result['Volume Capacity '].values / result['Capacity '].values)**result['Power'].values))
print(result.head())



NameError: name 'd_sol' is not defined

In [24]:
print(list(result.columns.values))
result['fft'] = result['Free Flow Time '].values
result['B'] = result['B'].values
result['Power'] = result['Power'].values
result['Volume Capacity '] = result['Volume Capacity '].values
net_tmp = result.drop(['Capacity ', 'Free Flow Time ', 'Cost '], axis=1)
print(net_tmp.head().to_csv())

f = open("data/" + network_name + "_bis_net.csv", "w")
f.write(net_tmp.to_csv())
f.close()

NameError: name 'result' is not defined

In [25]:
print(list(result.columns.values))
result['a0'] = result['Free Flow Time '].values
result['a1'] = 0
result['a2'] = 0
result['a3'] = 0
result['a4'] = result['B'].values * result['Free Flow Time '].values * ((1/result['Capacity '].values)**result['Power'])
net_tmp = result.drop(['Capacity ', 'Free Flow Time ', 'B', 'Power', 'a0', 'a1', 'a2', 'a3', 'a4', 'Init node ', 'Term node '], axis=1)
print(net_tmp.head().to_csv())

f = open("data/" + network_name + "_bis_result.csv", "w")
f.write(net_tmp.to_csv())
f.close()

NameError: name 'result' is not defined

In [26]:
import scipy.sparse
np_od = d_od.values
od_csr = scipy.sparse.csr_matrix(np_od)

f = open("data/" + network_name + "_bis_od.csv", "w")
f.write("O,D,Ton\n")
for od, dod in od_csr.todok().items():
    f.write(str(od[0]+1) + "," + str(od[1]+1) + "," + str(dod) + "\n")
f.close()

In [27]:
print(d_node.head())

NameError: name 'd_node' is not defined

In [28]:
f = open("data/" + network_name + "_bis_node.csv", "w")


f.write(d_node.to_csv())
f.close()


NameError: name 'd_node' is not defined